# ICC-based Feature Selection for Radiomics Reproducibility Assessment

## Overview
This notebook assesses the computational reproducibility of an automated radiomics pipeline using intraclass correlation coefficients (ICC).

## Method
- **Sample**: A subset of 40 patients (120 discs) randomly selected from the entire cohort
- **Pipeline**: Complete automated pipeline (TotalSpineSeg segmentation + PyRadiomics feature extraction) independently executed twice using identical settings
- **ICC Model**: Two-way random effects model for absolute agreement [ICC(2,1)]
- **Tool**: pingouin package (version 0.5.5) in Python
- **Selection Criterion**: Features with ICC > 0.75 (excellent reproducibility) are retained for subsequent analysis

## Reference
- Koo TK, Li MY. A Guideline of Selecting and Reporting Intraclass Correlation Coefficients for Reliability Research. J Chiropr Med. 2016;15(2):155-163.

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import pingouin as pg
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print(f"Pingouin version: {pg.__version__}")

## 2. Configuration

**Please modify the following parameters according to your data:**

In [ ]:
# =============================================================================
# Configuration - Please modify according to your data
# =============================================================================

# File paths for the two independent feature extraction runs
PATH_RUN1 = 'data/features_run1.xlsx'  # First extraction
PATH_RUN2 = 'data/features_run2.xlsx'  # Second extraction

# Non-feature columns (e.g., sample ID, labels)
# These columns will be excluded from ICC calculation
ID_COLUMNS = ['MASK', 'disc_degree']

# ICC threshold for feature selection
ICC_THRESHOLD = 0.75

# Output directory
OUTPUT_DIR = 'results/'

## 3. Load Data

In [ ]:
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load feature data from two independent runs
features_run1 = pd.read_excel(PATH_RUN1)
features_run2 = pd.read_excel(PATH_RUN2)

print(f"Run 1: {features_run1.shape[0]} samples, {features_run1.shape[1]} columns")
print(f"Run 2: {features_run2.shape[0]} samples, {features_run2.shape[1]} columns")

## 4. Data Preprocessing

In [ ]:
# Sort by ID column to ensure consistent sample order
if 'MASK' in features_run1.columns:
    features_run1 = features_run1.sort_values('MASK').reset_index(drop=True)
    features_run2 = features_run2.sort_values('MASK').reset_index(drop=True)

# Get feature columns (excluding ID columns)
feature_columns = [col for col in features_run1.columns if col not in ID_COLUMNS]

# Validation
assert features_run1.shape[0] == features_run2.shape[0], "Sample counts do not match!"
print(f"Number of samples: {features_run1.shape[0]}")
print(f"Number of features: {len(feature_columns)}")

## 5. Calculate ICC(2,1)

In [ ]:
def calculate_icc(feature_name, run1_data, run2_data):
    """
    Calculate ICC(2,1) for a single feature.
    
    ICC(2,1): Two-way random effects model for absolute agreement.
    
    Parameters
    ----------
    feature_name : str
        Name of the feature
    run1_data : array-like
        Feature values from first extraction
    run2_data : array-like
        Feature values from second extraction
    
    Returns
    -------
    dict : Dictionary containing ICC value and confidence intervals
    """
    n_subjects = len(run1_data)
    
    # Create long-format DataFrame for ICC calculation
    df_long = pd.DataFrame({
        'Subject': list(range(n_subjects)) * 2,
        'Rater': ['Run1'] * n_subjects + ['Run2'] * n_subjects,
        'Value': list(run1_data) + list(run2_data)
    })
    
    try:
        # Calculate ICC using pingouin
        icc_result = pg.intraclass_corr(
            data=df_long,
            targets='Subject',
            raters='Rater',
            ratings='Value'
        )
        
        # Extract ICC(2,1) results - "ICC2" corresponds to ICC(2,1)
        icc2_row = icc_result[icc_result['Type'] == 'ICC2']
        
        if len(icc2_row) > 0:
            return {
                'Feature': feature_name,
                'ICC': icc2_row['ICC'].values[0],
                'CI95_lower': icc2_row['CI95%'].values[0][0],
                'CI95_upper': icc2_row['CI95%'].values[0][1],
                'F': icc2_row['F'].values[0],
                'pval': icc2_row['pval'].values[0]
            }
    except Exception as e:
        print(f"Error calculating ICC for {feature_name}: {e}")
    
    return {
        'Feature': feature_name,
        'ICC': np.nan,
        'CI95_lower': np.nan,
        'CI95_upper': np.nan,
        'F': np.nan,
        'pval': np.nan
    }

In [ ]:
# Calculate ICC for all features
print("Calculating ICC(2,1) for all features...")
print("=" * 60)

icc_results = []

for i, feature in enumerate(feature_columns):
    run1_values = features_run1[feature].values
    run2_values = features_run2[feature].values
    
    # Handle constant features
    if np.std(run1_values) == 0 and np.std(run2_values) == 0:
        result = {
            'Feature': feature,
            'ICC': 1.0,
            'CI95_lower': 1.0,
            'CI95_upper': 1.0,
            'F': np.inf,
            'pval': 0.0
        }
    else:
        result = calculate_icc(feature, run1_values, run2_values)
    
    icc_results.append(result)
    
    if (i + 1) % 100 == 0 or (i + 1) == len(feature_columns):
        print(f"Progress: {i+1}/{len(feature_columns)}")

icc_df = pd.DataFrame(icc_results)
print("\nICC calculation completed!")

## 6. Results Summary

In [ ]:
print("ICC Statistics:")
print("=" * 60)
print(icc_df['ICC'].describe())

print("\nTop 20 features by ICC value:")
display(icc_df.sort_values('ICC', ascending=False).head(20))

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(icc_df['ICC'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(x=ICC_THRESHOLD, color='r', linestyle='--', linewidth=2, 
                label=f'Threshold ({ICC_THRESHOLD})')
axes[0].set_xlabel('ICC Value', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of ICC Values', fontsize=14)
axes[0].legend()

# Category bar chart
icc_categories = {
    'Poor\n(<0.5)': (icc_df['ICC'] < 0.5).sum(),
    'Moderate\n(0.5-0.75)': ((icc_df['ICC'] >= 0.5) & (icc_df['ICC'] < 0.75)).sum(),
    'Good\n(0.75-0.9)': ((icc_df['ICC'] >= 0.75) & (icc_df['ICC'] < 0.9)).sum(),
    'Excellent\n(>=0.9)': (icc_df['ICC'] >= 0.9).sum()
}

colors = ['#ff6b6b', '#ffd93d', '#6bcb77', '#4d96ff']
bars = axes[1].bar(icc_categories.keys(), icc_categories.values(), 
                   color=colors, edgecolor='black')
axes[1].set_xlabel('ICC Category', fontsize=12)
axes[1].set_ylabel('Number of Features', fontsize=12)
axes[1].set_title('Features by ICC Category', fontsize=14)

for bar, value in zip(bars, icc_categories.values()):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 str(value), ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}ICC_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Feature Selection (ICC > 0.75)

In [ ]:
# Select features with excellent reproducibility
selected_features_df = icc_df[icc_df['ICC'] > ICC_THRESHOLD].copy()
selected_features = selected_features_df['Feature'].tolist()

print(f"Feature Selection Results:")
print("=" * 60)
print(f"Original features: {len(feature_columns)}")
print(f"Features with ICC > {ICC_THRESHOLD}: {len(selected_features)}")
print(f"Retention rate: {len(selected_features)/len(feature_columns)*100:.2f}%")

## 8. Save Results

In [ ]:
# Save ICC results
icc_df.to_excel(f'{OUTPUT_DIR}ICC_results_all_features.xlsx', index=False)
selected_features_df.to_excel(f'{OUTPUT_DIR}ICC_selected_features.xlsx', index=False)
pd.DataFrame({'selected_features': selected_features}).to_csv(
    f'{OUTPUT_DIR}selected_feature_names.csv', index=False)

# Save filtered feature data
columns_to_keep = [col for col in ID_COLUMNS + selected_features if col in features_run1.columns]
features_filtered = features_run1[columns_to_keep].copy()
features_filtered.to_excel(f'{OUTPUT_DIR}features_after_ICC_selection.xlsx', index=False)

print("Results saved:")
print(f"  - {OUTPUT_DIR}ICC_results_all_features.xlsx")
print(f"  - {OUTPUT_DIR}ICC_selected_features.xlsx")
print(f"  - {OUTPUT_DIR}selected_feature_names.csv")
print(f"  - {OUTPUT_DIR}features_after_ICC_selection.xlsx")
print(f"  - {OUTPUT_DIR}ICC_distribution.png")

## 9. Summary Report

In [ ]:
print("\n" + "=" * 70)
print("                ICC Feature Selection Summary Report")
print("=" * 70)

print(f"\n[Data Overview]")
print(f"  - Number of samples: {features_run1.shape[0]}")
print(f"  - Original features: {len(feature_columns)}")

print(f"\n[ICC Method]")
print(f"  - ICC model: ICC(2,1) - Two-way random effects, absolute agreement")
print(f"  - Tool: pingouin v{pg.__version__}")
print(f"  - Selection threshold: ICC > {ICC_THRESHOLD}")

print(f"\n[ICC Statistics]")
print(f"  - Mean: {icc_df['ICC'].mean():.4f}")
print(f"  - Median: {icc_df['ICC'].median():.4f}")
print(f"  - Std: {icc_df['ICC'].std():.4f}")
print(f"  - Range: [{icc_df['ICC'].min():.4f}, {icc_df['ICC'].max():.4f}]")

print(f"\n[Feature Categories]")
print(f"  - Poor (ICC < 0.5): {(icc_df['ICC'] < 0.5).sum()}")
print(f"  - Moderate (0.5 <= ICC < 0.75): {((icc_df['ICC'] >= 0.5) & (icc_df['ICC'] < 0.75)).sum()}")
print(f"  - Good (0.75 <= ICC < 0.9): {((icc_df['ICC'] >= 0.75) & (icc_df['ICC'] < 0.9)).sum()}")
print(f"  - Excellent (ICC >= 0.9): {(icc_df['ICC'] >= 0.9).sum()}")

print(f"\n[Final Results]")
print(f"  - Selected features: {len(selected_features)}")
print(f"  - Retention rate: {len(selected_features)/len(feature_columns)*100:.2f}%")

print("\n" + "=" * 70)

---

## ICC Interpretation Guidelines (Koo & Li, 2016)

| ICC Value | Reliability |
|-----------|-------------|
| < 0.50 | Poor |
| 0.50 - 0.75 | Moderate |
| 0.75 - 0.90 | Good |
| >= 0.90 | Excellent |

## References
1. Koo TK, Li MY. A Guideline of Selecting and Reporting Intraclass Correlation Coefficients for Reliability Research. J Chiropr Med. 2016;15(2):155-163.
2. Shrout PE, Fleiss JL. Intraclass correlations: uses in assessing rater reliability. Psychol Bull. 1979;86(2):420-428.